In [1]:
import cv2
from matplotlib import pyplot as plt
import numpy as np

In [ ]:
cam = cv2.VideoCapture("./parkingLotVID.mp4")
slots = []
n = 20;
while True:
    _, im0 = cam.read()
    showCrosshair = False
    fromCenter = False
    if(n>0):
        r = cv2.selectROI("Image1", im0, fromCenter, showCrosshair)
        n = n-1
        slots.append(r)

    
    for slot in slots:
        im0 = cv2.rectangle(im0, (int(slot[0]),int(slot[1])), 
                                    (int(slot[0]+slot[2]),int(slot[1]+slot[3])), (255, 0, 0), )
    
    cv2.imshow("Image", im0)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
        
cam.release()
cv2.destroyAllWindows()

In [1]:
import pytesseract

In [5]:


pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

path = "C:/Users/asus\Desktop/Major/dataset/Indian Number Plate/Indian Number Plate/"

img_name = "KA67PN9942.png"

filename = path + img_name

gray = cv2.imread(filename, 0)
gray = cv2.resize( gray, None, fx = 3, fy = 3, interpolation = cv2.INTER_CUBIC)
blur = cv2.GaussianBlur(gray, (5,5), 0)
gray = cv2.medianBlur(gray, 3)
# perform otsu thresh (using binary inverse since opencv contours work better with white text)
ret, thresh = cv2.threshold(blur, 0, 255, cv2.THRESH_OTSU | cv2.THRESH_BINARY_INV)
cv2.imshow("Otsu", thresh)
cv2.waitKey(0)
rect_kern = cv2.getStructuringElement(cv2.MORPH_RECT, (3,3))

 
dilation = cv2.dilate(thresh, rect_kern, iterations = 1)

try:
    contours, hierarchy = cv2.findContours(dilation, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
except:
    ret_img, contours, hierarchy = cv2.findContours(dilation, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
sorted_contours = sorted(contours, key=lambda ctr: cv2.boundingRect(ctr)[0])

# create copy of image
im2 = gray.copy()

plate_num = ""
# loop through contours and find letters in license plate
for cnt in sorted_contours:
    x,y,w,h = cv2.boundingRect(cnt)
    height, width = im2.shape
    
    # if height of box is not a quarter of total height then skip
    if height / float(h) > 6: continue
    ratio = h / float(w)
    # if height to width ratio is less than 1.5 skip
    if ratio < 1.5: continue
    area = h * w
    # if width is not more than 25 pixels skip
    if width / float(w) > 15: continue
    # if area is less than 100 pixels skip
    if area < 100: continue
    # draw the rectangle
    rect = cv2.rectangle(im2, (x,y), (x+w, y+h), (0,255,0),2)
    roi = thresh[y-5:y+h+5, x-5:x+w+5]
    roi = cv2.bitwise_not(roi)
    roi = cv2.medianBlur(roi, 5)
    #cv2.imshow("ROI", roi)
    #cv2.waitKey(0)
    text = pytesseract.image_to_string(roi, config='-c tessedit_char_whitelist=0123456789ABCDEFGHIJKLMNOPQRSTUVWXYZ --psm 8 --oem 3')
    #print(text)
    plate_num += text
print(plate_num)
cv2.imshow("Character's Segmented", im2)
cv2.waitKey(0)
cv2.destroyAllWindows()

error: OpenCV(4.4.0) C:\Users\appveyor\AppData\Local\Temp\1\pip-req-build-52oirelq\opencv\modules\imgproc\src\median_blur.dispatch.cpp:283: error: (-215:Assertion failed) !_src0.empty() in function 'cv::medianBlur'


In [6]:
import requests
import base64
import json

In [13]:
def ocr(IMAGE_PATH):
	SECRET_KEY = 'sk_f52ed46c3a0902f92f82e967'
	with open(IMAGE_PATH, 'rb') as image_file:
    		img_base64 = base64.b64encode(image_file.read())

	url = 'https://api.openalpr.com/v2/recognize_bytes?recognize_vehicle=1&country=ind&secret_key=%s' % (SECRET_KEY)  #Replace 'ind' with  your country code
	r = requests.post(url, data = img_base64)
	try:
		return(r.json()['results'][0]['plate'])
		
	except:
		print("No number plate found")

In [14]:
path = "C:/Users/asus/Desktop/Major/dataset/Indian Plates/positive/"

img_name = "Audi-A4-528794d.jpg_0359_0305_0250_0204_0066.png"

filename = path + img_name

ocr(filename)

'KA03MN9993'